In [1]:
def train(train_loader, model, loss_fn, optimizer):

    # switch to train mode
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):

        prediction = model(inputs)
        loss = loss_fn(prediction, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def accuracy(val_loader, loss_fn, criterion):

    model.eval() 
    _return_ = 0
    for i, (inputs, labels) in enumerate(val_loader):

      prediction = model(inputs)
      batch_size = prediction.shape[0]
      _, prediction = prediction.topk(1, 1, True, True)
      prediction = prediction.t()
      correct = (prediction == labels).sum()
      print(correct / batch_size)
      _return_ += correct
    print(_return_ / (i + 1))

In [ ]:
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from google.colab import drive

drive.mount('/content/drive')

num_of_batch = 32
learning_rate = 0.01
model = models.resnet18(pretrained=True) #Take pretrained model from torchvision
num_ftrs = model.fc.in_features 
model.fc = nn.Linear(num_ftrs, 20) #Change the number of class to be 20

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                                momentum=0.9,
                                weight_decay=1e-4)

train_path = "/content/drive/My Drive/CSE48001/dataset/train/"
test_path = "/content/drive/My Drive/CSE48001/dataset/test/"

#Prepare the dataset for training
train_dataset = datasets.ImageFolder( 
    train_path,
    transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],
                                  [0.5, 0.5, 0.5]),
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=num_of_batch, shuffle=True,
    pin_memory=True, sampler=None)

#Prepare the dataset for testing
test_dataset = datasets.ImageFolder(
    test_path,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],
                                  [0.5, 0.5, 0.5]),
    ]))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=num_of_batch, shuffle=False,
    pin_memory=True, sampler=None)

for i in range(3):

    train(train_loader, model, loss_fn, optimizer)
    accuracy(test_loader, model, loss_fn)